# Trait `AutoInstantiable`

In [1]:
# import classes
from dataclasses import dataclass
from python_middlewareable import (
    RequestBase,
    MiddlewareBase,
    MiddlewareNextCallBase,
    MiddlewareableBase,
    DataStructurableRequestMixin,
    RequestDataBase,
    ResponseDataBase,
    TransportDataBase,
    AutoInstantiable,
)


# define the payload structure
@dataclass
class PayloadData(RequestDataBase):  # inherit from RequestDataBase
    name: str


# define the response structure
@dataclass
class ResponseData(ResponseDataBase):  # inherit from ResponseDataBase
    value: str = ""


# add the `DataStructurableRequestMixin` to the request class
@dataclass
class Request(
    DataStructurableRequestMixin[PayloadData, ResponseData, TransportDataBase],
    RequestBase,
):
    pass


# create a middleware
class OneMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> None:
        await next_call(request)

        request.response_data.value += request.request_data.name + " from OneMiddleware"


# add `AutoInstantiable` to the middlewareable
class App(
    AutoInstantiable[Request, ResponseData, TransportDataBase],
    MiddlewareableBase[Request],
):
    middlewares = [OneMiddleware]

    # same classes passed to "AutoInstantiable" above
    request_class = Request
    response_data_class = ResponseData
    transport_data_class = TransportDataBase


# middlewareable
app = App()

# process request
result = await app.process_middlewares_from_payload(data=PayloadData(name="John"))

# check the result
print(result)

# output:
# ResponseData(data='John from OneMiddleware')

ResponseData(value='John from OneMiddleware')
